In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/CyberSecurity/Task_3_TTPS/Complete_Data.csv')

In [ ]:
data=data[['Tactic','Technique','Procedure']]

In [ ]:
list1=data['Procedure'].tolist()

In [ ]:
list2=pd.read_csv("/content/drive/MyDrive/CyberSecurity/Task_3_TTPS/FineTunning.csv")['0'].tolist()

In [ ]:
paragraphs=list1+list2
labels_Attacks=[1 for i in range(0,len(paragraphs))]

In [ ]:
file_path = "/content/drive/MyDrive/CyberSecurity/Task_3_TTPS/RandomParagraphs.txt"
file = open(file_path, "r")
file_content = file.read()
file.close()

nonAttackParagraphs = file_content.split("\n\n")
labels_NAttacks=[0 for i in range(len(nonAttackParagraphs))]

In [ ]:
paragraphs=paragraphs+nonAttackParagraphs
labels=labels_Attacks+labels_NAttacks

In [ ]:

from joblib import dump
# Save the best model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.utils import shuffle
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

# Sample data


# Preprocessing steps
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    # Remove special characters and digits
    text = re.sub(r'\W+|\d+', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize text
    tokens = text.split()
    # Remove stop words and lemmatize tokens
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    # Join tokens back to text
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

# Shuffle the data
paragraphs, labels = shuffle(paragraphs, labels, random_state=42)

# Preprocess the text data
paragraphs = [preprocess_text(paragraph) for paragraph in paragraphs]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(paragraphs, labels, test_size=0.2, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', RandomForestClassifier())
])

# Define hyperparameters to tune
parameters = {
    'tfidf__max_features': [1000, 5000, 10000],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5, 10]
}

# Perform grid search for hyperparameter tuning
grid_search = GridSearchCV(pipeline, parameters, cv=5, scoring='f1')
grid_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)

# Get the best model from the grid search
best_model = grid_search.best_estimator_

# Evaluate on the test set
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))


In [ ]:
model_filename = "AttackOrNot.joblib"
dump(best_model, model_filename)

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
from nltk.stem import WordNetLemmatizer


In [ ]:
def summarize(text, per):
    nlp = spacy.load('en_core_web_sm')
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    return summary


In [ ]:
file_path = "/content/drive/MyDrive/CyberSecurity/Task_3_TTPS/threat.txt"
with open(file_path, "r", encoding="utf-8") as file:
    file_content = file.read()

testParaSummarization = file_content.split("\n\n")


In [ ]:
import re

from bs4 import BeautifulSoup

In [ ]:
def read_paragraphs_from_file(file_path):
    with open(file_path, 'r',encoding="UTF-8") as file:
        content = file.read()
        paragraphs = content.split('\n\n')
        paragraphs = [paragraph.strip() for paragraph in paragraphs if paragraph.strip() and len(paragraph.split())<=400]
    return paragraphs

# Provide the file path of the text file
file_path = '/content/drive/MyDrive/CyberSecurity/Task_3_TTPS/threat.txt'
paragraphs_clean = read_paragraphs_from_file(file_path)
paragraphs=paragraphs_clean.copy()



In [ ]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib
# Step 1: Load TTPs from the CSV file
ttps_df = pd.read_csv("/content/drive/MyDrive/CyberSecurity/Task_3_TTPS/Complete_Data.csv")
ttps = ttps_df["Procedure"].tolist()


# Step 2: Load paragraphs from the text file
# with open("threat.txt", "r") as file:
#     paragraphs = file.readlines()

# Step 3: Preprocess TTPs and paragraphs

# Preprocessing function for removing unnecessary characters or formatting
def preprocess_text(text):
    # Apply any preprocessing steps (e.g., removing special characters, lowercasing)
    string = re.sub(r'http\S+|www\S+', '', text)

    # Replace email addresses with 'email'
    email_regex = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b')
    string = email_regex.sub('email', string)

    # Replace IP addresses with 'ip'
    ip_regex = re.compile(r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b')
    string = ip_regex.sub('ip', string)

    # Remove newline characters
    string = string.replace('\n', '')
    soup = BeautifulSoup(string, "html.parser")
    string = soup.get_text()

    # Remove special characters
    string = re.sub(r"[^a-zA-Z0-9\s]", "", string)

    # Remove extra whitespaces
    string = re.sub(r"\s+", " ", string)

    preprocessed_text = string.strip()  # Remove leading/trailing whitespaces
    return preprocessed_text

preprocessed_ttps = [preprocess_text(ttp) for ttp in ttps]
preprocessed_paragraphs = [preprocess_text(paragraph) for paragraph in paragraphs]

# Step 4: Vectorize TTPs and paragraphs using TF-IDF

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the preprocessed TTPs
ttp_vectors = vectorizer.fit_transform(preprocessed_ttps)

# Transform the preprocessed paragraphs into vectors
paragraphs = [paragraph.strip() for paragraph in paragraphs]  # Remove leading/trailing whitespaces
paragraph_vectors = vectorizer.transform(preprocessed_paragraphs)

# Step 5: Generate sentence embeddings using Universal Sentence Encoder

# Load the Universal Sentence Encoder model
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Encode TTPs and paragraphs to get sentence embeddings
ttp_embeddings = use_model(preprocessed_ttps)
paragraph_embeddings = use_model(preprocessed_paragraphs)

# Step 6: Find the most similar TTP for each paragraph
#joblib.dump(ttp_embeddings, "ttp_embeddings.pkl")

most_similar_ttps = []
most_similar_indexes=[]
# Iterate over each paragraph
for i in range(len(paragraphs)):
    paragraph_embedding = paragraph_embeddings[i]

    # Compute the cosine similarity between the paragraph embedding and all TTP embeddings
    similarities = cosine_similarity([paragraph_embedding], ttp_embeddings)

    # Find the index of the most similar TTP
    most_similar_index = np.argmax(similarities)

    # Retrieve the most similar TTP
    most_similar_ttp = ttps[most_similar_index]
    most_similar_ttps.append(most_similar_ttp)
    most_similar_indexes.append(most_similar_index)




In [ ]:
import joblib

In [ ]:
paragraphs

In [ ]:
attackModel=joblib.load("/content/drive/MyDrive/CyberSecurity/Task_3_TTPS/AttackOrNot.joblib")


In [ ]:

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def most_similar_string(sentence, string_list):
  """
  Returns the most similar string in the string list to the given sentence.

  Args:
    sentence: The sentence to compare.
    string_list: A list of strings.

  Returns:
    The most similar string in the string list.
  """

  tfidf_vectorizer = TfidfVectorizer()
  tfidf_matrix = tfidf_vectorizer.fit_transform(string_list + [sentence])

  use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
  sentence_embeddings = use_model([sentence])[0]
  string_embeddings = use_model(string_list)

  cosine_similarities = cosine_similarity([sentence_embeddings], string_embeddings)[0]

  most_similar_index = np.argmax(cosine_similarities)
  return string_list[most_similar_index]


In [ ]:
attackOrNot_predict=attackModel.predict(paragraphs)

In [ ]:
for i in range(len(paragraphs)):
  if(attackOrNot_predict[i]==0):
    continue

  print("\nParagraph ",i," ==>\n",paragraphs[i])
  print("Tactic    :",ttps_df.iloc[most_similar_indexes[i]][1])
  print("Techique  :",ttps_df.iloc[most_similar_indexes[i]][2])
  print("Procedure :",ttps_df.iloc[most_similar_indexes[i]][3])
  print("Summarization --->",summarize(paragraphs[i],0.3))

  sentences=paragraphs[i].split('.')
  techniques=ttps_df.iloc[most_similar_indexes[i]][2]
  procedure=ttps_df.iloc[most_similar_indexes[i]][3]
  print("Sentences for TTPs identification :")
  print("=========>>",most_similar_string(techniques,sentences))
  print("=========>>",most_similar_string(procedure,sentences))
  print("------------------------------------------------")


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import joblib
import re
from bs4 import BeautifulSoup
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow_hub as hub
from heapq import nlargest
from string import punctuation

In [ ]:


class TTPS_prediction:

    def __init__(self):
        self.ttps_Data = pd.read_csv("/content/drive/MyDrive/CyberSecurity/Task_3_TTPS/Complete_Data.csv")
        self.sentenceEncoder_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
        self.nlp = spacy.load('en_core_web_sm')
        self.attackOrNot_Model = joblib.load("/content/drive/MyDrive/CyberSecurity/Task_3_TTPS/AttackOrNot.joblib")
        self.ttp_embeddings = joblib.load("/content/drive/MyDrive/CyberSecurity/Task_3_TTPS/ttp_embeddings.pkl")
        self.vectorizer = TfidfVectorizer()

    def read_paragraphs_from_file(self,file_path):
        try:
          with open(file_path, 'r',encoding="UTF-8") as file:
              content = file.read()
              paragraphs = content.split('\n\n')
              paragraphs = [paragraph.strip() for paragraph in paragraphs if paragraph.strip() and len(paragraph.split())<=400]
          self.vectorizationAndEmbedding(paragraphs)
        except :
          print("XXXXXXXXXXXXXX File is not present at give path XXXXXXXXXXX")



    def preprocess_text(self, text):
        # Apply any preprocessing steps (e.g., removing special characters, lowercasing)
        string = re.sub(r'http\S+|www\S+', '', text)

        # Replace email addresses with 'email'
        email_regex = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b')
        string = email_regex.sub('email', string)

        # Replace IP addresses with 'ip'
        ip_regex = re.compile(r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b')
        string = ip_regex.sub('ip', string)

        # Remove newline characters
        string = string.replace('\n', '')
        soup = BeautifulSoup(string, "html.parser")
        string = soup.get_text()

        # Remove special characters
        string = re.sub(r"[^a-zA-Z0-9\s]", "", string)

        # Remove extra whitespaces
        string = re.sub(r"\s+", " ", string)

        preprocessed_text = string.strip()  # Remove leading/trailing whitespaces
        return preprocessed_text

    def summarize(self, text, per):
        doc = self.nlp(text)
        tokens = [token.text for token in doc]
        word_frequencies = {}
        for word in doc:
            if word.text.lower() not in list(STOP_WORDS):
                if word.text.lower() not in punctuation:
                    if word.text not in word_frequencies.keys():
                        word_frequencies[word.text] = 1
                    else:
                        word_frequencies[word.text] += 1
        max_frequency = max(word_frequencies.values())
        for word in word_frequencies.keys():
            word_frequencies[word] = word_frequencies[word] / max_frequency
        sentence_tokens = [sent for sent in doc.sents]
        sentence_scores = {}
        for sent in sentence_tokens:
            for word in sent:
                if word.text.lower() in word_frequencies.keys():
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word.text.lower()]
                    else:
                        sentence_scores[sent] += word_frequencies[word.text.lower()]
        select_length = int(len(sentence_tokens) * per)
        summary = nlargest(select_length, sentence_scores, key=sentence_scores.get)
        final_summary = [word.text for word in summary]
        summary = ''.join(final_summary)
        return summary

    def most_similar_string(self, sentence, string_list):
        """
        Returns the most similar string in the string list to the given sentence.

        Args:
          sentence: The sentence to compare.
          string_list: A list of strings.

        Returns:
          The most similar string in the string list.
        """

        tfidf_matrix = self.vectorizer.transform(string_list + [sentence])

        sentence_embeddings = self.sentenceEncoder_model([sentence])[0]
        string_embeddings = self.sentenceEncoder_model(string_list)

        cosine_similarities = cosine_similarity([sentence_embeddings], string_embeddings)[0]

        most_similar_index = np.argmax(cosine_similarities)
        return string_list[most_similar_index]

    def vectorizationAndEmbedding(self, paragraphs):
        preprocessed_paragraphs = [self.preprocess_text(paragraph) for paragraph in paragraphs]

        # Fit the vectorizer on the preprocessed paragraphs
        self.vectorizer.fit(preprocessed_paragraphs)

        # Transform the preprocessed paragraphs into vectors

        paragraph_vectors = self.vectorizer.transform(preprocessed_paragraphs)
        paragraph_embeddings = self.sentenceEncoder_model(preprocessed_paragraphs)

        most_similar_ttps = []
        most_similar_indexes = []
        for i in range(len(paragraphs)):
            paragraph_embedding = paragraph_embeddings[i]

            # Compute the cosine similarity between the paragraph embedding and all TTP embeddings
            similarities = cosine_similarity([paragraph_embedding], self.ttp_embeddings)

            # Find the index of the most similar TTP
            most_similar_index = np.argmax(similarities)

            # Retrieve the most similar TTP
            most_similar_ttp = self.ttps_Data.iloc[most_similar_index]
            most_similar_ttps.append(most_similar_ttp)
            most_similar_indexes.append(most_similar_index)

        # Print the most similar TTPs for each paragraph
        for i in range(len(paragraphs)):
            if self.attackOrNot_Model.predict([paragraphs[i]])[0] == 0:
                continue

            print("\nParagraph ", i, " =========\n", paragraphs[i])
            print("Tactic    :", self.ttps_Data.iloc[most_similar_indexes[i]][1])
            print("Technique :", self.ttps_Data.iloc[most_similar_indexes[i]][2])
            print("Procedure :", self.ttps_Data.iloc[most_similar_indexes[i]][3])
            print("Summarization --->", self.summarize(paragraphs[i], 0.3))

            sentences = paragraphs[i].split('.')
            technique = self.ttps_Data.iloc[most_similar_indexes[i]][2]
            procedure = self.ttps_Data.iloc[most_similar_indexes[i]][3]
            print("Sentences for TTPs identification :")
            most_similar_technique=self.most_similar_string(technique, sentences)
            if(most_similar_technique!=""):
              print("=========>>", most_similar_technique)

            print("=========>>", self.most_similar_string(procedure, sentences))
            print("------------------------------------------------")

    def Main(self):
      print("Enter your choice ")
      print("1:To load .txt file")
      print("2:To load paragraph")
      print("Press other key to exit")
      choice=int(input("Input choice: "))
      while(choice==1 or choice==2):
        if(choice==1):
          location=input("Enter the location of file : ")
          self.read_paragraphs_from_file(location)
        else:
          string=str(input("Enter the paragraph : "))
          self.vectorizationAndEmbedding([string])

        choice=int(input("Enter the choice :"))






In [ ]:
x = TTPS_prediction()

In [ ]:
x.Main()